In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, QuantileTransformer
from sklearn.neighbors import LocalOutlierFactor

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.ensemble import StackingRegressor, AdaBoostRegressor

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, PowerTransformer, OneHotEncoder
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error

In [2]:
cowpea = pd.read_excel('../data/prepared.xlsx', sheet_name='Cowpea')    ; cowpea['crop']   = 'cowpea'
maize = pd.read_excel('../data/prepared.xlsx', sheet_name='Maize')      ; maize['crop']    = 'maize'
rice = pd.read_excel('../data/prepared.xlsx', sheet_name='Rice')        ; rice['crop']     = 'rice'
chickpea = pd.read_excel('../data/prepared.xlsx', sheet_name='Chickpea'); chickpea['crop'] = 'chickpea'
mustard = pd.read_excel('../data/prepared.xlsx', sheet_name='Mustard')  ; mustard['crop']  = 'mustard'

In [3]:
data = pd.concat([cowpea, rice, maize, chickpea, mustard], axis=0).reset_index(drop=True)

In [4]:
clf = LocalOutlierFactor(n_neighbors=20)

new_data = []
org_cols = data.columns
for i, outlier_label in enumerate(clf.fit_predict(data[['GSR', 'CT']])):
    if outlier_label==1:
        new_data.append(data.iloc[i,:])
data = pd.DataFrame(new_data, columns=org_cols)

In [5]:
data = data[(data['GSR']<300) | (data['Rn']>150)]
data = data[(data['Rn']<500) | (data['crop']!='cowpea')]
data = data[(data['Rn']<400) | (data['GSR']>500) | (data['crop']!='rice')]
data = data[(data['Rn']<300) | (data['GSR']>375)]

In [6]:
data.loc[:, 'Time'] = data.loc[:, 'Time'].apply(lambda x: x.hour)

In [7]:
data.loc[:, 'timesin'] = np.sin(data.loc[:, 'Time'] * (2 * np.pi) / 12)
data.loc[:, 'timecos'] = np.cos(data.loc[:, 'Time'] * (2 * np.pi) / 12)

In [8]:
# ohe = OneHotEncoder(sparse=False)
# df = ohe.fit_transform(data['crop'].values.reshape(-1,1))

# print(df.shape)
# df = pd.DataFrame(df, columns=ohe.get_feature_names(['crop']))
df = pd.get_dummies(data[['crop']])
data = pd.concat([df, data], axis=1)

In [9]:
scalerx = StandardScaler()
scalery = StandardScaler()
data[['GSR','CT',]] = scalerx.fit_transform(data[['GSR','CT']])
data[['Rn']] = scalery.fit_transform(data[['Rn']])

In [10]:
data.columns

Index(['crop_chickpea', 'crop_cowpea', 'crop_maize', 'crop_mustard',
       'crop_rice', 'Date', 'Time', 'GSR', 'CT', 'Rn', 'crop', 'ST_5cm',
       'ST_10cm', 'ST_15cm', 'timesin', 'timecos'],
      dtype='object')

In [11]:
feature_cols = [c for c in data.columns if c not in ['ST_5cm','ST_10cm','ST_15cm','Date','Time','crop','Rn']]
X = data[feature_cols]
y = data['Rn']

In [12]:
estimators = [
    ('GBR', GradientBoostingRegressor(random_state=42)),
    ('RF', RandomForestRegressor(random_state=42)),
    ('Ridge pipeline', Pipeline(steps=[
#         ('powert', PowerTransformer()),
        ('poly2', PolynomialFeatures(degree=2, interaction_only=True)),
        ('ridgef', Ridge(alpha=50, random_state=42))
    ])),
    ('ridge', Ridge(alpha=40, random_state=42)),
    ('SVR', SVR(kernel='rbf', gamma='auto', C=0.05)),
    ('adaboost', AdaBoostRegressor())
]

In [13]:
# Use KFold croos validation
kfold = KFold(n_splits=5)

In [14]:
# Train all models
def train(estimators, X, y, cv, scoring, verbose):
    if verbose:
        print("Scoring criteria:", str(scoring))
        print("CV:", cv)
        print("y std:", np.std(y))
        print('\n')
    for model in estimators if isinstance(estimators, list) else [estimators]:
        model[1].fit(X, y)
        cross_scores = cross_val_score(model[1], X, y, scoring=scoring, cv=cv)
        if verbose:
            print(model[0], "mean cv score:", np.mean(cross_scores))
            print(model[0], "all cv scores:", cross_scores)
            print('\n')

In [15]:
all_mse = {}
all_rmse = {}
for model in estimators:
    all_mse[model[0]] = []
    all_rmse[model[0]] = []
for (t_, v_) in kfold.split(X, y):
    train(estimators=estimators, X=X.iloc[t_], y=y.iloc[t_], cv=5, scoring='neg_root_mean_squared_error', verbose=0)
    for model in estimators:
        y_pred = scalery.inverse_transform(model[1].predict(X.iloc[v_][feature_cols]).reshape(-1,1))
        y_true = scalery.inverse_transform(y.iloc[v_].values.reshape(-1,1))
        
        mse = mean_squared_error(y_true, y_pred)
        rmse = np.sqrt(mse)
        
        all_mse[model[0]].extend([mse])
        all_rmse[model[0]].extend([rmse])

In [16]:
for model in estimators:
    print(model[0],":")
    print("All folds MSE :", all_mse[model[0]])
    print("All folds RMSE :", all_rmse[model[0]])
    print("Mean MSE :", np.mean(all_mse[model[0]]))
    print("Mean RMSE :", np.mean(all_rmse[model[0]]))
    print("\n")

GBR :
All folds MSE : [5219.026349359445, 11783.826737626172, 4586.932721068801, 3100.6668608609225, 2310.8598352962954]
All folds RMSE : [72.24282905146673, 108.55333591201227, 67.72689806176568, 55.6836318935908, 48.07140350870042]
Mean MSE : 5400.2625008423265
Mean RMSE : 70.45561968550717


RF :
All folds MSE : [3906.8322886621413, 11040.482670037767, 4661.574062083717, 2514.747887415392, 2320.22786228427]
All folds RMSE : [62.5046581357113, 105.07370113419327, 68.27572088292966, 50.147262013148755, 48.16874362368475]
Mean MSE : 4888.7729540966575
Mean RMSE : 66.83401715793354


Ridge pipeline :
All folds MSE : [4500.631340086857, 15084.087761950641, 3714.4086045151284, 4398.81994912804, 3839.9865503351402]
All folds RMSE : [67.08674489112478, 122.81729422988703, 60.94594822065802, 66.32360024250825, 61.96762501770695]
Mean MSE : 6307.5868412031605
Mean RMSE : 75.828242520377


ridge :
All folds MSE : [3343.0357624780163, 10930.911415926375, 3794.6581801916836, 3852.01219490431, 29

## Stacked estimator

In [17]:
stacked_estimator = StackingRegressor(
    estimators=estimators,
    final_estimator=RandomForestRegressor(random_state=42)
)

In [18]:
cross_scores = cross_val_score(
    stacked_estimator,
    X,
    y,
    scoring='neg_root_mean_squared_error',
    cv=5
)

print("Stacked estimator mean cv score:", np.mean(cross_scores))
print("Stacked estimator all cv scores:", cross_scores)

Stacked estimator mean cv score: -0.5050362031926395
Stacked estimator all cv scores: [-0.46445612 -0.78159813 -0.47275788 -0.4221128  -0.38425609]


In [19]:
all_stacked_mses = []
all_stacked_rmses = []
for (t_, v_) in kfold.split(X, y):
    stacked_estimator.fit(X.iloc[t_], y.iloc[t_])
    y_pred = scalery.inverse_transform(stacked_estimator.predict(X.iloc[v_][feature_cols]).reshape(-1,1))
    y_true = scalery.inverse_transform(y.iloc[v_].values.reshape(-1,1))
    
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    
    all_stacked_mses.append(mse)
    all_stacked_rmses.append(rmse)
    print("Stacked estimator MSE:", mse)
    print("Stacked estimator RMSE:", rmse)
    print("\n")

print("Stacked estimator mean MSE:", np.mean(all_stacked_mses))
print("Stacked estimator mean RMSE:", np.mean(all_stacked_rmses))

Stacked estimator MSE: 4509.8043997660225
Stacked estimator RMSE: 67.15507724488165


Stacked estimator MSE: 11637.930165315978
Stacked estimator RMSE: 107.87923880578681


Stacked estimator MSE: 4434.400908152341
Stacked estimator RMSE: 66.59129754068726


Stacked estimator MSE: 3447.0075352536614
Stacked estimator RMSE: 58.71122154455366


Stacked estimator MSE: 2627.8320032751058
Stacked estimator RMSE: 51.26238390160085


Stacked estimator mean MSE: 5331.395002352622
Stacked estimator mean RMSE: 70.31984380750205
